In [4]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import codecs
!wget https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
import json 

from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium 
print('Complete')

--2020-08-25 17:03:04--  https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
Resolving en.wikipedia.org (en.wikipedia.org)... 208.80.153.224, 2620:0:860:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.153.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 56520 (55K) [text/html]
Saving to: ‘List_of_postal_codes_of_Canada:_M’

100%[======================================>] 56,520      --.-K/s   in 0.004s  

2020-08-25 17:03:04 (14.6 MB/s) - ‘List_of_postal_codes_of_Canada:_M’ saved [56520/56520]

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge


# Toronto Data

In [5]:
toronto = codecs.open('List_of_postal_codes_of_Canada:_M', encoding='utf-8')
html = toronto.read()
soup = BeautifulSoup(html, 'html.parser')
toronto_df = (pd.read_html(str(soup.table))[0])
toronto_df.drop(axis=0, index=0, inplace=True)
toronto_df

,Postal Code,Borough,Neighbourhood
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
10,M2B,Not assigned,Not assigned


# Organize and Clean Data

In [6]:
toronto_columns = ['PostalCode', 'Borough', 'Neighbourhood'] 

#remove not assigned
toronto_df = toronto_df[toronto_df['Borough'] != "Not assigned"].reset_index(drop=True) 
toronto_df

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


# Get Longitude and Latitude

In [7]:
!wget http://cocl.us/Geospatial_data

--2020-08-25 17:10:39--  http://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.55.161.7
Connecting to cocl.us (cocl.us)|169.55.161.7|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data [following]
--2020-08-25 17:10:39--  https://cocl.us/Geospatial_data
Connecting to cocl.us (cocl.us)|169.55.161.7|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-08-25 17:10:40--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-08-25 17:10:40--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mv

In [8]:
toronto_geo = pd.read_csv('http://cocl.us/Geospatial_data')
toronto_geo.head()
toronto_geo.columns = ['Postal Code', 'Latitude', 'Longitude']
toronto_geo

#merge data
toronto_merged_df = pd.merge(toronto_df, toronto_geo, on= 'Postal Code')
toronto_merged_df


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# Foursquare API Information

In [9]:
CLIENT_ID = 'LPJNPAKLUKGOPLMLQK5VN4IIXDGZG4TD1G4CIRUZ2041M32T' # your Foursquare ID
CLIENT_SECRET = 'NDO54RFAMEAFLB3J2QRSB0B5K5DM3OI5DRXATAYQCWGG3XQN' # your Foursquare Secret
VERSION = '20200815' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LPJNPAKLUKGOPLMLQK5VN4IIXDGZG4TD1G4CIRUZ2041M32T
CLIENT_SECRET:NDO54RFAMEAFLB3J2QRSB0B5K5DM3OI5DRXATAYQCWGG3XQN


In [10]:
toronto_merged_df.loc[2, 'Neighbourhood']
neighbourhood_lat = toronto_merged_df.loc[2, 'Latitude'] # neighborhood latitude value
neighbourhood_long = toronto_merged_df.loc[2, 'Longitude'] # neighborhood longitude value

neighbourhood_name = toronto_merged_df.loc[2, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_lat, 
                                                               neighbourhood_long))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


In [11]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius
 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_lat, 
    neighbourhood_long, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=LPJNPAKLUKGOPLMLQK5VN4IIXDGZG4TD1G4CIRUZ2041M32T&client_secret=NDO54RFAMEAFLB3J2QRSB0B5K5DM3OI5DRXATAYQCWGG3XQN&v=20200815&ll=43.6542599,-79.3606359&radius=500&limit=100'

In [12]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f45462f0c92d673f998b4ff'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Corktown',
  'headerFullLocation': 'Corktown, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 44,
  'suggestedBounds': {'ne': {'lat': 43.6587599045, 'lng': -79.3544279001486},
   'sw': {'lat': 43.6497598955, 'lng': -79.36684389985142}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '54ea41ad498e9a11e9e13308',
       'name': 'Roselle Desserts',
       'location': {'address': '362 King St E',
        'crossStreet': 'Trinity St',
        'lat': 43.653446723052674,
        'lng': -79.3620167174383,
        'labeledLatLngs': [{'label': 'display',
 

# Category Types

In [13]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
venues = results['response']['groups'][0]['items']
    
toronto_nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
toronto_nearby_venues = toronto_nearby_venues.loc[:, filtered_columns]

# filter the category for each row
toronto_nearby_venues['venue.categories'] = toronto_nearby_venues.apply(get_category_type, axis=1)

# clean columns
toronto_nearby_venues.columns = [col.split(".")[-1] for col in toronto_nearby_venues.columns]

toronto_nearby_venues

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Body Blitz Spa East,Spa,43.654735,-79.359874
4,Impact Kitchen,Restaurant,43.656369,-79.356980
5,Corktown Common,Park,43.655618,-79.356211
6,Dominion Pub and Kitchen,Pub,43.656919,-79.358967
7,Morning Glory Cafe,Breakfast Spot,43.653947,-79.361149
8,The Distillery Historic District,Historic Site,43.650244,-79.359323
9,The Extension Room,Gym / Fitness Center,43.653313,-79.359725


In [15]:
print('{} venues were returned by Foursquare.'.format(toronto_nearby_venues.shape[0]))

44 venues were returned by Foursquare.


In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
toronto_venues = getNearbyVenues(names=toronto_merged_df['Neighbourhood'],
                                   latitudes=toronto_merged_df['Latitude'],
                                   longitudes=toronto_merged_df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [20]:
toronto_venues

,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
5,Victoria Village,43.725882,-79.315572,The Frig,43.727051,-79.317418,French Restaurant
6,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place
7,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
8,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
9,"Regent Park, Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center


In [21]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

There are 269 uniques categories.


,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
1,"Alderwood, Long Branch",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
4,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
5,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
6,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
7,"Brockton, Parkdale Village, Exhibition Place",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
8,"Business reply mail Processing Centre, South C...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.058824
9,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.058824,0.058824,0.058824,0.058824,0.176471,0.117647,0.000000,0.000000,...,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


In [23]:
num_top_venues = 10

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0             Clothing Store  0.25
1             Breakfast Spot  0.25
2  Latin American Restaurant  0.25
3                     Lounge  0.25
4          Accessories Store  0.00
5                     Museum  0.00
6              Movie Theater  0.00
7                      Motel  0.00
8        Moroccan Restaurant  0.00
9        Monument / Landmark  0.00


----Alderwood, Long Branch----
                 venue  freq
0          Pizza Place  0.29
1          Coffee Shop  0.14
2       Sandwich Place  0.14
3         Skating Rink  0.14
4                  Gym  0.14
5                  Pub  0.14
6               Museum  0.00
7        Movie Theater  0.00
8                Motel  0.00
9  Moroccan Restaurant  0.00


----Bathurst Manor, Wilson Heights, Downsview North----
                 venue  freq
0          Coffee Shop  0.10
1                 Bank  0.10
2             Pharmacy  0.05
3          Bridal Shop  0.05
4            Pet Store  0.05
5    Mobile P

                  venue  freq
0              Pharmacy  0.14
1                Bakery  0.14
2               Brewery  0.07
3  Gym / Fitness Center  0.07
4                  Bank  0.07
5           Supermarket  0.07
6                  Café  0.07
7           Music Venue  0.07
8         Grocery Store  0.07
9                  Park  0.07


----East Toronto, Broadview North (Old East York)----
                             venue  freq
0                             Park  0.50
1                Convenience Store  0.25
2                      Coffee Shop  0.25
3       Modern European Restaurant  0.00
4                           Museum  0.00
5                    Movie Theater  0.00
6                            Motel  0.00
7              Moroccan Restaurant  0.00
8              Monument / Landmark  0.00
9  Molecular Gastronomy Restaurant  0.00


----Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood----
                venue  freq
0         Pizza Place  0.12
1         Coffee Shop  0.12
2      

                             venue  freq
0                      Swim School  0.33
1                             Park  0.33
2                         Bus Line  0.33
3                Accessories Store  0.00
4       Modern European Restaurant  0.00
5                            Motel  0.00
6              Moroccan Restaurant  0.00
7              Monument / Landmark  0.00
8  Molecular Gastronomy Restaurant  0.00
9                Mobile Phone Shop  0.00


----Leaside----
                    venue  freq
0     Sporting Goods Shop  0.09
1             Coffee Shop  0.09
2           Shopping Mall  0.06
3  Furniture / Home Store  0.06
4                    Bank  0.06
5            Burger Joint  0.06
6       Fish & Chips Shop  0.03
7          Sandwich Place  0.03
8           Smoothie Shop  0.03
9      Mexican Restaurant  0.03


----Little Portugal, Trinity----
                   venue  freq
0                    Bar  0.09
1            Coffee Shop  0.07
2       Asian Restaurant  0.07
3  Vietnamese Restau

                             venue  freq
0                     Home Service   0.5
1                              Bar   0.5
2                Accessories Store   0.0
3  Molecular Gastronomy Restaurant   0.0
4                           Museum   0.0
5                    Movie Theater   0.0
6                            Motel   0.0
7              Moroccan Restaurant   0.0
8              Monument / Landmark   0.0
9       Modern European Restaurant   0.0


----Runnymede, Swansea----
                venue  freq
0                Café  0.09
1         Pizza Place  0.09
2    Sushi Restaurant  0.06
3         Coffee Shop  0.06
4                 Pub  0.06
5  Italian Restaurant  0.06
6   Health Food Store  0.03
7        Gourmet Shop  0.03
8       Burrito Place  0.03
9        Dessert Shop  0.03


----Runnymede, The Junction North----
                        venue  freq
0                    Bus Line  0.33
1           Convenience Store  0.33
2              Breakfast Spot  0.33
3           Accessories Stor

              venue  freq
0  Ramen Restaurant  0.09
1              Café  0.06
2       Coffee Shop  0.06
3  Sushi Restaurant  0.06
4        Restaurant  0.06
5    Sandwich Place  0.06
6       Pizza Place  0.06
7   Bubble Tea Shop  0.03
8         Pet Store  0.03
9         Juice Bar  0.03


----Willowdale, Willowdale West----
                             venue  freq
0                      Pizza Place  0.17
1                      Coffee Shop  0.17
2                             Bank  0.17
3                          Butcher  0.17
4                    Grocery Store  0.17
5                         Pharmacy  0.17
6               Miscellaneous Shop  0.00
7                Mobile Phone Shop  0.00
8       Modern European Restaurant  0.00
9  Molecular Gastronomy Restaurant  0.00


----Woburn----
                             venue  freq
0                      Coffee Shop  0.67
1                Korean Restaurant  0.33
2  Molecular Gastronomy Restaurant  0.00
3                           Museum  0.00
4  

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 10

indicators = ['st', 'nd', 'rd']


columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Latin American Restaurant,Breakfast Spot,Lounge,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
1,"Alderwood, Long Branch",Pizza Place,Gym,Sandwich Place,Skating Rink,Coffee Shop,Pub,Yoga Studio,Doner Restaurant,Discount Store,Distribution Center
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pet Store,Ice Cream Shop,Shopping Mall,Sandwich Place,Diner,Middle Eastern Restaurant,Restaurant,Mobile Phone Shop
3,Bayview Village,Japanese Restaurant,Café,Chinese Restaurant,Bank,Yoga Studio,Distribution Center,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Greek Restaurant,Café,Japanese Restaurant,Juice Bar,Liquor Store,Butcher,Restaurant


In [25]:
#clusters
from sklearn.cluster import KMeans
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [26]:
neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Clothing Store,Latin American Restaurant,Breakfast Spot,Lounge,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
1,"Alderwood, Long Branch",Pizza Place,Gym,Sandwich Place,Skating Rink,Coffee Shop,Pub,Yoga Studio,Doner Restaurant,Discount Store,Distribution Center
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pet Store,Ice Cream Shop,Shopping Mall,Sandwich Place,Diner,Middle Eastern Restaurant,Restaurant,Mobile Phone Shop
3,Bayview Village,Japanese Restaurant,Café,Chinese Restaurant,Bank,Yoga Studio,Distribution Center,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Greek Restaurant,Café,Japanese Restaurant,Juice Bar,Liquor Store,Butcher,Restaurant


In [27]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [28]:
toronto_merged = toronto_merged_df
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood', how= 'right')

toronto_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3,Park,Food & Drink Shop,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Ethiopian Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,1,Portuguese Restaurant,Pizza Place,Hockey Arena,French Restaurant,Coffee Shop,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Park,Bakery,Café,Pub,Breakfast Spot,Theater,Beer Store,Shoe Store,Restaurant
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,1,Clothing Store,Furniture / Home Store,Accessories Store,Event Space,Vietnamese Restaurant,Boutique,Coffee Shop,Shoe Store,Athletics & Sports,Drugstore
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Yoga Studio,Persian Restaurant,Bar,Beer Bar,Smoothie Shop,Sandwich Place,Café,Chinese Restaurant,Portuguese Restaurant
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353,1,Print Shop,Fast Food Restaurant,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
7,M3B,North York,Don Mills,43.745906,-79.352188,1,Gym,Japanese Restaurant,Restaurant,Beer Store,Coffee Shop,Asian Restaurant,Supermarket,Italian Restaurant,Café,Discount Store
13,M3C,North York,Don Mills,43.725900,-79.340923,1,Gym,Japanese Restaurant,Restaurant,Beer Store,Coffee Shop,Asian Restaurant,Supermarket,Italian Restaurant,Café,Discount Store
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1,Pizza Place,Intersection,Pharmacy,Gym / Fitness Center,Breakfast Spot,Gastropub,Athletics & Sports,Bank,Café,Donut Shop
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Japanese Restaurant,Cosmetics Shop,Ramen Restaurant,Fast Food Restaurant,Hotel,Bookstore


# Toronto Map

In [29]:
# create map
map_clusters = folium.Map(location=[43.651070, -79.347015], zoom_start=12)


x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

# Korean Data

In [37]:
!wget https://en.wikipedia.org/wiki/List_of_postal_codes_in_South_Korea

--2020-08-25 18:45:09--  https://en.wikipedia.org/wiki/List_of_postal_codes_in_South_Korea
Resolving en.wikipedia.org (en.wikipedia.org)... 208.80.153.224, 2620:0:860:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.153.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 89673 (88K) [text/html]
Saving to: ‘List_of_postal_codes_in_South_Korea.1’

100%[======================================>] 89,673      --.-K/s   in 0.007s  

2020-08-25 18:45:09 (12.9 MB/s) - ‘List_of_postal_codes_in_South_Korea.1’ saved [89673/89673]



In [42]:
code = codecs.open('List_of_postal_codes_in_South_Korea', encoding='utf-8')
html = code.read()
soup = BeautifulSoup(k_html, 'html.parser')
k_code_df = (pd.read_html(str(soup.table))[0])
k_code_df.drop(axis=0, index=0, inplace=True)
k_code_df

,Code,Division name,Hangul,Hanja
,Seoul Special City,Seoul Special City,Seoul Special City,Seoul Special City
1,013NN ~ 015NN,Dobong District,도봉구,道峰區
2,015NN ~ 019NN,Nowon District,노원구,蘆原區
3,020NN ~ 023NN,Jungnang District,중랑구,中浪區
4,024NN ~ 026NN,Dongdaemun District,동대문구,東大門區
5,027NN ~ 029NN,Seongbuk District,성북구,城北區
6,030NN ~ 032NN,Jongno District,종로구,鍾路區
7,033NN ~ 035NN,Eunpyeong District,은평구,恩平區
8,036NN ~ 038NN,Seodaemun District,서대문구,西大門區
9,039NN ~ 042NN,Mapo District,마포구,麻浦區


# Cleaning Korean Data

In [66]:
import json
import urllib
import requests

where = urllib.parse.quote_plus("""
{
    "postalCode": {
        "$exists": true
    },
    "placeName": {
        "$exists": true
    },
    "geoPosition": {
        "$exists": true
    },
    "countryCode": {
        "$exists": true
    }
}
""")
url = 'https://parseapi.back4app.com/classes/KR?count=1&limit=10&where=%s' % where
headers = {
    'X-Parse-Application-Id': 'zS2XAEVEZAkD081UmEECFq22mAjIvX2IlTYaQfai', # This is the fake app's application id
    'X-Parse-Master-Key': 't6EjVCUOwutr1ruorlXNsH3Rz65g0jiVtbILtAYU' # This is the fake app's readonly master key
}
data = json.loads(requests.get(url, headers=headers).content.decode('utf-8')) # Here you have the data that you need
print(json.dumps(data, indent=2))



{
  "results": [
    {
      "objectId": "vd73E1p4wc",
      "geoPosition": {
        "__type": "GeoPoint",
        "latitude": 33.9626,
        "longitude": 126.2965
      },
      "countryCode": "KR",
      "postalCode": "63000",
      "placeName": "\ucd94\uc790\uba74",
      "adminName1": "\uc81c\uc8fc\ud2b9\ubcc4\uc790\uce58\ub3c4",
      "adminCode1": "01",
      "adminName2": "\uc81c\uc8fc\uc2dc",
      "adminCode2": "39010",
      "adminName3": "\ucd94\uc790\uba74",
      "adminCode3": "3901032",
      "accuracy": "6",
      "createdAt": "2020-02-24T14:41:55.809Z",
      "updatedAt": "2020-02-24T14:41:55.809Z"
    },
    {
      "objectId": "LDs8NIUlt3",
      "geoPosition": {
        "__type": "GeoPoint",
        "latitude": 33.9469,
        "longitude": 126.3247
      },
      "countryCode": "KR",
      "postalCode": "63001",
      "placeName": "\ucd94\uc790\uba74",
      "adminName1": "\uc81c\uc8fc\ud2b9\ubcc4\uc790\uce58\ub3c4",
      "adminCode1": "01",
      "adminName2": 